In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import experimenter
import pandas as pd 
import matplotlib.pyplot as plt 

sns.set_theme()

In [ ]:
# Maximum number of replicas that the system can manage.
MAX_REPLICA_COUNT = 100

# Number of experiments
NUMBER_OF_EXPERIMENTS = 5

# Each experiment will be done by this many times for consistency.
EXPERIMENT_PROOF = 5

# Int array of replicas for each experiment
REPLICAS: np.ndarray = experimenter.calculate_number_of_replicas(MAX_REPLICA_COUNT, NUMBER_OF_EXPERIMENTS)

# How much should we wait for the kubernetes api to be stabilized
COOLDOWNS: np.ndarray = 2 * np.ones((NUMBER_OF_EXPERIMENTS,), dtype=int) + (REPLICAS * 0.004).astype(int)

# Namespace of the experimentation
NAMESPACE = "experiments"

# Driver used in kubectl
DRIVER = "kwok"

# Framework
FRAMEWORK = "native"

In [ ]:
def draw_replicas(replicas=REPLICAS):
    plt.figure(figsize=(8, 4))
    plt.plot(range(len(replicas)), replicas)
    plt.title("Replica Count by Experiments")
    plt.xlabel("Experiment")
    plt.ylabel("Replicas")
    plt.show()

draw_replicas()
print(COOLDOWNS)
print(REPLICAS)

In [ ]:
# Do the experiment
# experimenter.experiment_with(FRAMEWORK, DRIVER, 1, [1, 1], [1, 1])
experimenter.experiment_with(FRAMEWORK, DRIVER, EXPERIMENT_PROOF, REPLICAS.tolist(), COOLDOWNS.tolist())

In [ ]:
KNATIVE_EXPERIMENT_NAME = "exps/20230228_"

def process_experiment(filename):
    data_df = pd.read_feather(filename)

    data_df["api_server_creation_delta"] = data_df["event_created_at"] - data_df["before_create_timestamp"]
    data_df["api_server_deletion_delta"] = data_df["event_deleted_at"] - data_df["before_delete_timestamp"]
    
    data_df.to_feather(filename)
    return data_df

knative_minikube_df = process_experiment(KNATIVE_EXPERIMENT_NAME)
knative_minikube_df

In [ ]:
# BETTER PROCESS THIS DATA
plt.figure(figsize=(20, 4))
plt.subplot(1, 2, 1)
plt.plot(knative_minikube_df["api_server_deletion_delta"]); plt.title("Creation Times")
plt.subplot(1, 2, 2)
plt.plot(knative_minikube_df["api_server_creation_delta"]); plt.title("Deletion Times")
plt.show()